In [1]:
import torch

# 파이토치 기초 문법

In [2]:
torch.tensor([[1., -1.], [1., -1.]])

tensor([[ 1., -1.],
        [ 1., -1.]])

In [3]:
# print(torch.tensor([[1,2], [3,4]]))
# print(torch.tensor([[1,2], [3,4]], device='cuda:0') )
print(torch.tensor([[1,2], [3,4]], dtype=torch.float64) )

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)


In [6]:
temp = torch.tensor([[1, 2], [3, 4]])

print(temp.shape)
print('---------------------')
print(temp.view(4,1))
print('---------------------')
print(temp.view(-1))
print('---------------------')
print(temp.view(1,-1))
print('---------------------')
print(temp.view(-1,1))

torch.Size([2, 2])
---------------------
tensor([[1],
        [2],
        [3],
        [4]])
---------------------
tensor([1, 2, 3, 4])
---------------------
tensor([[1, 2, 3, 4]])
---------------------
tensor([[1],
        [2],
        [3],
        [4]])


# MINIST 데이터셋

In [10]:
import torch.nn as nn

model = nn.Linear(in_features=1, out_features=1, bias=True)

In [12]:
class MLP(nn.Module):
    def __init__(self, inputs):
        super(MLP, self).__init__()
        self.layer = nn.Linear(inputs, 1) # 계층 정의
        self.activation = nn.sigmoid # 활성화함수 정의
    
    def forward(self, X):
        X = self.layer(X)
        X = self.activation(X)
        return X

In [13]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=30, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(in_features=30*5*5, out_features=10, bias=True),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        x = self.layer3(x)
        return x
    
model = MLP()

print('printing Children\n---------------------')
print(list(model.children()))
print('printing Modules\n---------------------')
print(list(model.modules()))

printing Children
---------------------
[Sequential(
  (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
), Sequential(
  (0): Linear(in_features=750, out_features=10, bias=True)
  (1): ReLU(inplace=True)
)]
printing Modules
---------------------
[MLP(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 30, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=750, o

## 모델 학습

In [1]:
# 라이브러리 호출
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import requests

In [2]:
# 하이퍼파라미터 설정
batch_size = 64
epoch = 10
lr = 0.001

In [3]:
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1.0,)) # 평균이 0.5, 표준편차가 1.0이 되도록 데이터의 분포(normalize)를 조정
])

download_root = '/home/xogns5037/Basic_PyTorch/data'

# 훈련 & 검증 & 데이터셋 
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

# 데이터 로더 (train과 valid, test 배치 사이즈 달라도 크게 상관없음)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
# 모델링
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # layer 정의
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28) # 2차원 -> 1차원으로 변환
        # x = self.fc1(x)
        # x = torch.relu(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [5]:
# 모델 생성
model = SimpleNN()

In [6]:
# optimizer, loss function 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [7]:
# 모델 학습(CPU)
for i in range(epoch):
    for images, labels in train_loader:
        optimizer.zero_grad() # 기울기 초기화
        preds = model(images) # 모델 예측
        loss = loss_fn(preds, labels) # loss 계산
        loss.backward() # 역전파
        optimizer.step() # 가중치 업데이트
    
    print(f'Epoch {i+1}, Loss: {loss.item():.4f}')

Epoch 1, Loss: 0.1300
Epoch 2, Loss: 0.4743
Epoch 3, Loss: 0.0360
Epoch 4, Loss: 0.0998
Epoch 5, Loss: 0.0652
Epoch 6, Loss: 0.0525
Epoch 7, Loss: 0.1958
Epoch 8, Loss: 0.0729
Epoch 9, Loss: 0.0075
Epoch 10, Loss: 0.0165


In [13]:
# 모델 학습(GPU)

# GPU 디바이스 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 모델, 손실 함수, 옵티마이저를 디바이스로 이동
model = SimpleNN().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for i in range(epoch):
    model.train()  # 모델을 학습 모드로 설정
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # 기울기 초기화
        preds = model(images)  # 모델 예측
        loss = loss_fn(preds, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트
    
    # 에포크당 손실 출력
    print(f'Epoch {i+1}, Loss: {loss.item():.4f}')


Epoch 1, Loss: 0.3778
Epoch 2, Loss: 0.1148
Epoch 3, Loss: 0.0784
Epoch 4, Loss: 0.0409
Epoch 5, Loss: 0.0130
Epoch 6, Loss: 0.0856
Epoch 7, Loss: 0.0879
Epoch 8, Loss: 0.0384
Epoch 9, Loss: 0.1699
Epoch 10, Loss: 0.0072


## 모델 평가

In [10]:
# model valid
model.eval()  # 모델을 평가 모드로 설정 = 파라미터 업데이트 X
total = 0
correct = 0

# GPU 디바이스 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in test_loader:
        preds = model(images) # 모델 예측
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0) # 전체 샘플 수
        correct += (predicted == labels).sum().item()

accuracy = (correct / total) * 100
print(f'Accuracy of the model on test dataset: {accuracy:.2f}%')

Accuracy of the model on test dataset: 96.86%


In [13]:
# 한 epoch이 끝날때마다 valuation을 수행하고, 전체 수행 후 test data에 대한 accuracy를 출력

# GPU 디바이스 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 모델, 손실 함수, 옵티마이저를 디바이스로 이동
model = SimpleNN().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
for i in range(epoch):
    model.train()  # 모델을 학습 모드로 설정
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # 기울기 초기화
        preds = model(images)  # 모델 예측
        loss = loss_fn(preds, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 가중치 업데이트
    
    # 에포크당 validation 수행
    with torch.no_grad():  # 기울기 계산 비활성화
        model.eval()
        total = 0
        correct = 0
        
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            preds = model(images)
            _, predicted = torch.max(preds.data, 1)            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        accuracy = (correct / total) * 100
        print(f'Epoch {i+1}, Loss: {loss.item():.4f}, Val Accuracy: {accuracy:.2f}%')
        
# 모델 평가
model.eval()  # 모델을 평가 모드로 설정 = 파라미터 업데이트 X
total = 0
correct = 0

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        preds = model(images) # 모델 예측
        _, predicted = torch.max(preds.data, 1)
        total += labels.size(0) # 전체 샘플 수
        correct += (predicted == labels).sum().item()
        
accuracy = (correct / total) * 100
print(f'Accuracy of the model on test dataset: {accuracy:.2f}%')

Epoch 1, Loss: 0.1889, Accuracy: 92.26%
Epoch 2, Loss: 0.2389, Accuracy: 94.11%
Epoch 3, Loss: 0.1773, Accuracy: 95.80%
Epoch 4, Loss: 0.2362, Accuracy: 96.37%
Epoch 5, Loss: 0.0435, Accuracy: 96.51%
Epoch 6, Loss: 0.0077, Accuracy: 97.06%
Epoch 7, Loss: 0.0129, Accuracy: 96.75%
Epoch 8, Loss: 0.0216, Accuracy: 97.06%
Epoch 9, Loss: 0.0816, Accuracy: 97.27%
Epoch 10, Loss: 0.0522, Accuracy: 97.23%
Accuracy of the model on test dataset: 97.23%


# 합성곱 신경망

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
train_dataset = torchvision.datasets.FashionMNIST(root='/home/xogns5037/Basic_PyTorch/data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.FashionMNIST(root='/home/xogns5037/Basic_PyTorch/data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

100.0%


Extracting /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw



100.0%


Extracting /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw



100.0%


Extracting /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw



100.0%

Extracting /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/xogns5037/Basic_PyTorch/data/FashionMNIST/raw



In [5]:
train_loader = DataLoader(dataset=train_dataset, batch_size=100)
test_loader = DataLoader(dataset=test_dataset, batch_size=100)

In [6]:
labels_map = {0 : 'T-Shirt', 1 : 'Trouser', 2 : 'Pullover', 3 : 'Dress', 4 : 'Coat', 5 : 'Sandal', 6 : 'Shirt', 7 : 'Sneaker', 8 : 'Bag', 9 : 'Ankle Boot'}

## DNN vs CNN

In [7]:
# DNN

class FashionDNN(nn.Module):
    def __init__(self):
        super(FashionDNN, self).__init__()
        self.fc1 = nn.Linear(in_features=28*28, out_features=256)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=256, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)
        
    def forward(self, input_data):
        out = input_data.view(-1, 28*28)
        out = F.relu(self.fc1(out))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [8]:
learning_rate = 0.001
model = FashionDNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

FashionDNN(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [9]:
num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        train = Variable(images.view(100, 1, 28, 28)) 
        labels = Variable(labels)
        # Variable은 torch.Tensor를 감싸고, 계산 그래프를 만들어줌, 하지만 0.4.0 버전 이후로는 불필요
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
        
        if not (count % 50):
            total = 0
            correct = 0
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
                test = Variable(images.view(100, 1, 28, 28))
                
                outputs = model(test)
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
                total += len(labels)
                
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            
        if not (count % 500):
            print(f'Iteration: {count}, Loss: {loss.data}, Accuracy: {accuracy}')
        

Iteration: 500, Loss: 0.5764753818511963, Accuracy: 83.27999877929688
Iteration: 1000, Loss: 0.44199612736701965, Accuracy: 84.91999816894531
Iteration: 1500, Loss: 0.3234631419181824, Accuracy: 84.47000122070312
Iteration: 2000, Loss: 0.37893372774124146, Accuracy: 85.50999450683594
Iteration: 2500, Loss: 0.2274029403924942, Accuracy: 85.86000061035156
Iteration: 3000, Loss: 0.2534821927547455, Accuracy: 87.00999450683594


In [10]:
# CNN

class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [11]:
learning_rate = 0.001
model = FashionCNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [ ]:
num_epochs = 5
count = 5
loss_list = []
iteration_list = []
accuracy_list = [] 

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        count += 1
        
        if not (count % 50):
            total = 0
            correct = 0
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
                test = Variable(images.view(100, 1, 28, 28))
                outputs = model(test)
                predictions = torch.max(outputs, 1)[1].to(device)
                predictions_list.append(predictions)
                correct += (predictions == labels).sum()
                total += len(labels)
                
            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
            
        if not (count % 500):
            print(f'Iteration: {count}, Loss: {loss.data}, Accuracy: {accuracy}')

Iteration: 500, Loss: 0.22716039419174194, Accuracy: 89.54000091552734
Iteration: 1000, Loss: 0.1563497930765152, Accuracy: 90.79999542236328
Iteration: 1500, Loss: 0.26320797204971313, Accuracy: 90.55999755859375
Iteration: 2000, Loss: 0.17619465291500092, Accuracy: 90.80999755859375
Iteration: 2500, Loss: 0.20892852544784546, Accuracy: 91.39999389648438
Iteration: 3000, Loss: 0.14919129014015198, Accuracy: 89.65999603271484
